# AutoPrognosis classification

Welcome! This tutorial will walk you through the steps of selecting a model for a classification task using AutoPrognosis.

### Setup

In [ ]:
# stdlib
import json
import warnings

# third party
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

### Import ClassifierStudy

ClassifierStudy is the engine that learns an ensemble of pipelines and their hyperparameters automatically.

In [ ]:
# autoprognosis absolute
from autoprognosis.studies.classifiers import ClassifierStudy

### Load the target dataset

AutoPrognosis expects pandas.DataFrames as input.

For this example, we will use the [Breast Cancer Wisconsin Dataset](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)).

In [ ]:
# stdlib
from pathlib import Path

X, Y = load_breast_cancer(return_X_y=True, as_frame=True)

df = X.copy()
df["target"] = Y

### Create the classifier

While AutoPrognosis provides default plugins, it allows the user to customize the plugins for the pipelines.

You can see the supported plugins below:

In [ ]:
# List the available plugins

# autoprognosis absolute
from autoprognosis.plugins import Plugins

print(json.dumps(Plugins().list_available(), indent=2))

We will set a few custom plugins for the pipelines and create the classifier study.

In [ ]:
workspace = Path("workspace")
workspace.mkdir(parents=True, exist_ok=True)

study_name = "classification_example"

study = ClassifierStudy(
    study_name=study_name,
    dataset=df,  # pandas DataFrame
    target="target",  # the label column in the dataset
    num_iter=2,  # DELETE THIS LINE FOR BETTER RESULTS. how many trials to do for each candidate. Default: 50
    num_study_iter=1,  # DELETE THIS LINE FOR BETTER RESULTS. how many outer iterations to do. Default: 5
    classifiers=[
        "logistic_regression",
        "lda",
        "qda",
    ],  # DELETE THIS LINE FOR BETTER RESULTS.
    workspace=workspace,
)

### Search for the optimal ensemble


In [ ]:
study.run()

In [ ]:
# stdlib
import pprint

# autoprognosis absolute
from autoprognosis.utils.serialization import load_model_from_file
from autoprognosis.utils.tester import evaluate_estimator

output = workspace / study_name / "model.p"

model = load_model_from_file(output)

metrics = evaluate_estimator(model, X, Y)

print(f"Model {model.name()} ")
print("Score: ")

pprint.pprint(metrics)

## Serialization

In [ ]:
# autoprognosis absolute
from autoprognosis.utils.serialization import load_from_file, save_to_file

out = workspace / "tmp.bkp"
# Fit the model
model.fit(X, Y)

# Save
save_to_file(out, model)

# Reload
loaded_model = load_from_file(out)

print(loaded_model.name())

assert loaded_model.name() == model.name()

out.unlink()

## Congratulations!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement towards Machine learning and AI for medicine, you can do so in the following ways!

### Star AutoPrognosis on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the tools we're building.

- [Star AutoPrognosis](https://github.com/vanderschaarlab/autoprognosis)
- [Star HyperImpute](https://github.com/vanderschaarlab/hyperimpute)
